In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
import datetime as dt

#plot
import matplotlib.pyplot as plt
import seaborn as sns

## Download data

In [3]:
df_scans = pd.read_excel(r'D:\vm_exchange\PhD_Data\GantnerValidation\validation_scan.xlsx', index_col=False,
                        parse_dates=['Date'])
df_scans['TrackingTag'] = df_scans['TrackingTag'].astype(str)
df_scans['obsZone'] = df_scans['obsZone'].map(lambda x: 'zone_'+str(x).lower())
print(df_scans.shape)
df_scans.head(3)

(215, 9)


Date      Time  Pen Side Bird obsZone Position / Comments TrackingTag  \
0 2020-07-31  10:12:40    9   TS   sp  zone_4              Stange          69   
1 2020-07-31  10:12:40    9   TS   gp  zone_4              Stange         106   
2 2020-07-31  10:13:54    8   TS   bb  zone_4                 NaN          30   

   trackZone  
0        NaN  
1        NaN  
2        NaN

In [4]:
li_df_logs = []
for path_ in glob.glob(r'D:\vm_exchange\PhD_Data\GantnerValidation\log_00000001_pen*.csv'):
    df_logs = pd.read_csv(path_, sep=';', names=['Date','Time','tagID','TrackingTag', 'Zone','SignalStrength','_','__'],
                          parse_dates=['Date']) 
    df_logs['Time'] = df_logs['Time'].map(lambda x: dt.datetime.strptime(x, "%H:%M:%S"))
    #extract time from day
    df_logs['Time'] = df_logs['Time'].map(lambda x: dt.datetime.time(x))
    df_logs['TrackingTag'] = df_logs['TrackingTag'].astype(str)
    li_df_logs.append(df_logs[df_logs['Date']>dt.datetime(2020,7,29)])
df_logs = pd.concat(li_df_logs)
print(df_logs.shape)
df_logs.head(3)

C:\Users\camil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(24071, 8)


Date      Time     tagID TrackingTag                    Zone  \
91067 2020-07-30  14:37:31   2001558          58                  Tier 1   
91068 2020-07-30  14:37:32   2001044          44  Tier 3 Rampe + Nestbox   
91069 2020-07-30  14:37:36  0200151C          35        Tier 2 (mini 12)   

       SignalStrength  _  __  
91067               9  2   1  
91068              14  2   3  
91069              11  2   8

In [4]:
dico_trackzone_obszone = {'Tier 1':'zone_1', 
                          'Tier 3 Rampe + Nestbox':'zone_3', 
                          'Tier 2 (mini 12)':'zone_2',
                          'Tier 2 (mini 11)':'zone_2', 
                          'Wintergarten':'zone_wg', 
                          'Tier 2 (mini 10)':'zone_2',
                          'Tier 2 + untere Stange':'zone_2', 
                          'Tier 4 + obere Stange':'zone_4',
                          'Tier 2 (mini 4)':'zone_2', 
                          'Tier 2 (mini 5)':'zone_2', 
                          'Tier 2 (mini 3)':'zone_2',
                          'Tier 2 (mini 8)':'zone_2', 
                          'Tier 2 (mini 9)':'zone_2'}
df_logs['Zone'] = df_logs['Zone'].map(lambda x: dico_trackzone_obszone[x])
df_logs['Zone'].unique()

array(['zone_1', 'zone_3', 'zone_2', 'zone_wg', 'zone_4'], dtype=object)

#not good to use as changed over time a lot between pens, birds were moved etc
df_BP_TT = pd.read_excel(r'D:\vm_exchange\PhD_Data\GantnerValidation\BackpackID_matching_trackingTag.xlsx', sep=';', 
                         index_col=False)
print(df_BP_TT.shape)
df_BP_TT.head(3)

## Matching backpach & trackingTag

In [5]:
df_logs.iloc[0]['Time'], df_logs.iloc[0]['Date']

(datetime.time(14, 37, 31), Timestamp('2020-07-30 00:00:00'))

In [6]:
dt.time(14, 37, 31)<dt.time(14, 37, 30)

False

In [7]:
df_scans.iloc[0]['Time'], df_scans.iloc[0]['Date']

(datetime.time(10, 12, 40), Timestamp('2020-07-31 00:00:00'))

In [8]:
df_logs.head(2)

Date      Time    tagID TrackingTag    Zone  SignalStrength  _  __
91067 2020-07-30  14:37:31  2001558          58  zone_1               9  2   1
91068 2020-07-30  14:37:32  2001044          44  zone_3              14  2   3

In [9]:
df_scans.head(2)

Date      Time  Pen Side Bird obsZone Position / Comments TrackingTag  \
0 2020-07-31  10:12:40    9   TS   sp  zone_4              Stange          69   
1 2020-07-31  10:12:40    9   TS   gp  zone_4              Stange         106   

   trackZone  
0        NaN  
1        NaN

In [10]:
def bird_zone(df_logs, trackingtag, date, time, zone=None, print_=True):
    
    '''Print where the chicken was before and after this exact time and compute the the duration if a zone is indicated and 
    if the chicken is in this zone and output where he is now'''
    
    d = dt.date(1, 1, 1)
    df_ = df_logs[(df_logs['Date']==date) & (df_logs['TrackingTag']==trackingtag)]
    #remove same time
    df_ = df_.drop_duplicates(subset=None, keep='first', inplace=False)
    df_.index = df_['Time']
    if df_.shape[0]==0:
        if print_:
            print('No data for this chicken!')
        return 0,0,0,0
    try:
        x0 = df_.iloc[df_.index.get_loc(time,method='ffill')]
        x1 = df_.iloc[df_.index.get_loc(time,method='bfill')]
        #display(x0)
        #display(x1)
        t0 = 0 ; t1 = 0
        if zone!=None:
            if x0['Zone']==zone:
                t0 = dt.datetime.combine(d, time) - dt.datetime.combine(d, x0['Time'])
                t1 = dt.datetime.combine(d, x1['Time']) - dt.datetime.combine(d, time)
                if print_:
                    t = 'The chicken %s was %s already in that zone and staid %s more in this zone'%(trackingtag,
                                                                                                     str(t0),str(t1))
                    print(t)
                return t0.seconds,t1.seconds,x0['Zone'],t
        if print_:
            t = 'The chicken %s was in %s at %s and in %s at %s, and not in %s at %s'%(trackingtag, 
                                                                                         str(x0['Zone']),x0['Time'],
                                                                                         str(x1['Zone']),x1['Time'],
                                                                                         zone, time)
            print(t)
            return 0,0,x0['Zone'],t
    except Exception as e:
        pass
    return 0,0,0,0
    
#small e.g.
#t0,t1 = bird_zone(df_logs,'68',dt.datetime(2020,7,30),dt.time(15,30,30), 'zone_1')
#t0,t1 = bird_zone(df_logs,'68',dt.datetime(2020,7,30),dt.time(15,30,30), 'zone_wg')

In [11]:
df_scans['tuple_secondsbefore_after'] = df_scans.apply(lambda x: str(bird_zone(df_logs,x['TrackingTag'],x['Date'],x['Time'],
                                                                  x['obsZone'],print_=True)), axis=1)

The chicken 69 was 0:52:46 already in that zone and staid 0:07:33 more in this zone
The chicken 106 was 1:44:43 already in that zone and staid 0:01:50 more in this zone
The chicken 30 was 1:30:28 already in that zone and staid 2:10:59 more in this zone
The chicken 37 was 0:03:43 already in that zone and staid 0:04:28 more in this zone
The chicken 56 was in zone_2 at 10:15:19 and in zone_2 at 10:21:32, and not in zone_1 at 10:15:54
The chicken 56 was 0:01:43 already in that zone and staid 0:04:30 more in this zone
The chicken 5 was 0:07:34 already in that zone and staid 0:21:58 more in this zone
The chicken 37 was 0:00:27 already in that zone and staid 0:00:18 more in this zone
The chicken 45 was 0:03:15 already in that zone and staid 0:08:20 more in this zone
The chicken 30 was 1:40:18 already in that zone and staid 2:01:09 more in this zone
The chicken 37 was 0:01:17 already in that zone and staid 0:05:55 more in this zone
The chicken 9 was 0:26:38 already in that zone and staid 0:14:

The chicken 14 was 0:00:49 already in that zone and staid 0:00:23 more in this zone
The chicken 91 was 0:09:23 already in that zone and staid 0:03:22 more in this zone
The chicken 14 was in zone_2 at 15:30:20 and in zone_1 at 15:31:31, and not in zone_1 at 15:31:15
The chicken 42 was 0:22:47 already in that zone and staid 0:42:46 more in this zone
The chicken 91 was 0:00:15 already in that zone and staid 0:00:57 more in this zone
The chicken 64 was 0:18:14 already in that zone and staid 0:01:02 more in this zone
The chicken 64 was 0:00:04 already in that zone and staid 0:01:20 more in this zone
The chicken 64 was in zone_3 at 15:36:43 and in zone_4 at 15:38:07, and not in zone_4 at 15:38:00
The chicken 78 was 0:03:22 already in that zone and staid 0:00:50 more in this zone
The chicken 21 was 0:18:26 already in that zone and staid 0:10:40 more in this zone
The chicken 2 was 0:07:53 already in that zone and staid 0:11:53 more in this zone
The chicken 12 was 0:02:43 already in that zone a

In [12]:
#Note: each time obsZone !=trackZone, we should look if it due to system delay etc
df_scans['trackZone'] = df_scans['tuple_secondsbefore_after'].map(lambda x: eval(x)[2])
df_scans['automatic_comment'] = df_scans['tuple_secondsbefore_after'].map(lambda x: eval(x)[3])
df_scans['SecondsBefore'] = df_scans['tuple_secondsbefore_after'].map(lambda x: eval(x)[0])
df_scans['SecondsAfter'] = df_scans['tuple_secondsbefore_after'].map(lambda x: eval(x)[1])
df_scans['is_correct'] = df_scans.apply(lambda x: x['obsZone']==x['trackZone'], axis=1)
df_scans = df_scans.drop(['tuple_secondsbefore_after'], axis=1) 
df_scans

Date      Time  Pen Side Bird  obsZone  \
0   2020-07-31  10:12:40    9   TS   sp   zone_4   
1   2020-07-31  10:12:40    9   TS   gp   zone_4   
2   2020-07-31  10:13:54    8   TS   bb   zone_4   
3   2020-07-31  10:15:54    9   TS   sb   zone_1   
4   2020-07-31  10:15:54    9   TS   pb   zone_1   
5   2020-07-31  10:17:02    9   TS   pb   zone_2   
6   2020-07-31  10:19:19    8   TS   gg   zone_1   
7   2020-07-31  10:21:19    9   TS   sb   zone_2   
8   2020-07-31  10:21:40    9   TS  sws   zone_1   
9   2020-07-31  10:23:44    8   TS   bb   zone_4   
10  2020-07-31  10:24:05    9   TS   sb   zone_1   
11  2020-07-31  10:26:40    8   TS   sp   zone_1   
12  2020-07-31  10:26:40    8   TS   sg   zone_1   
13  2020-07-31  10:26:40    8   TS   wb   zone_1   
14  2020-07-31  10:30:20    8   TS   wp   zone_3   
15  2020-07-31  10:30:26    8   TS   wp   zone_3   
16  2020-07-31  10:31:31    8   TS   wp   zone_3   
17  2020-07-31  10:32:39    8   TS   sp   zone_1   
18  2020-07-31  10:32:10    9   TS  sws   zone_2   
19  2020-07-31  10:32:10    9   TS   sb   zone_2   
20  2020-07-31  10:52:36    9   TS   wb   zone_3   
21  2020-07-31  10:53:45    9   TS   wb   zone_3   
22  2020-07-31  10:36:00    8   TS   wp   zone_2   
23  2020-07-31  10:06:02    8   WG   gp   zone_2   
24  2020-07-31  10:07:42    8   WG   sg  zone_wg   
25  2020-07-31  10:09:39    8   WG   sg  zone_wg   
26  2020-07-31  10:09:39    8   WG   gg  zone_wg   
27  2020-07-31  10:09:39    8   WG   wp  zone_wg   
28  2020-07-31  10:12:30    9   WG  sws   zone_1   
29  2020-07-31  10:12:30    9   WG   bb   zone_1   
..         ...       ...  ...  ...  ...      ...   
185 2020-07-30  15:41:41    3   TS   wg   zone_4   
186 2020-07-30  15:41:28    4   TS   gb   zone_4   
187 2020-07-30  15:42:41    3   TS   pp   zone_2   
188 2020-07-30  15:54:35    5   TS  sws   zone_3   
189 2020-07-30  15:54:43    5   TS  sws   zone_3   
190 2020-07-30  15:55:05    5   TS  sws   zone_3   
191 2020-07-30  15:56:12    4   TS   gp   zone_3   
192 2020-07-30  15:56:34    4   TS   gp   zone_3   
193 2020-07-30  15:56:59    4   TS   gp   zone_3   
194 2020-07-30  15:58:40    3   TS   ss   zone_3   
195 2020-07-30  15:59:15    3   TS   ss   zone_3   
196 2020-07-30  15:59:36    3   TS   ss   zone_3   
197 2020-07-30  15:29:55    3   TS   pp   zone_1   
198 2020-07-30  15:31:15    3   TS   ss   zone_1   
199 2020-07-30  15:33:04    3   TS   ps   zone_1   
200 2020-07-30  15:34:55    3   TS   pp   zone_2   
201 2020-07-30  15:35:41    3   TS   wg   zone_1   
202 2020-07-30  15:36:47    3   TS   wg   zone_3   
203 2020-07-30  15:38:00    3   TS   wg   zone_4   
204 2020-07-30  15:38:55    3   TS   sb   zone_2   
205 2020-07-30  15:47:11    5   TS   SG   zone_1   
206 2020-07-30  15:47:11    5   TS   WS   zone_1   
207 2020-07-30  15:47:11    5   TS  GS    zone_1   
208 2020-07-30  15:48:23    5   TS   SS   zone_1   
209 2020-07-30  15:48:46    5   TS   GG   zone_1   
210 2020-07-30  15:49:20    5   TS   PB   zone_1   
211 2020-07-30  15:49:59    5   TS   WG   zone_1   
212 2020-07-30  15:50:38    5   TS   BS   zone_4   
213 2020-07-30  15:51:34    5   TS   GP   zone_2   
214 2020-07-30  15:51:34    5   TS   PP   zone_2   

                  Position / Comments TrackingTag trackZone  \
0                              Stange          69    zone_4   
1                              Stange         106    zone_4   
2                                 NaN          30    zone_4   
3                                 NaN          37    zone_1   
4    just before transition to zone 2          56    zone_2   
5                                 NaN          56    zone_2   
6                                 NaN           5    zone_1   
7                                 NaN          37    zone_2   
8                                 NaN          45    zone_1   
9                                 NaN          30    zone_4   
10                                NaN          37    zone_1   
11                           

In [13]:
df_scans['is_correct'].value_counts(normalize=True)

True     0.772093
False    0.227907
Name: is_correct, dtype: float64

In [14]:
df_scans.groupby(['is_correct','obsZone'])['Time'].count().reset_index()

is_correct  obsZone  Time
0       False   zone_1    11
1       False   zone_2     5
2       False   zone_3    15
3       False   zone_4    12
4       False  zone_wg     6
5        True   zone_1    90
6        True   zone_2    25
7        True   zone_3    16
8        True   zone_4    18
9        True  zone_wg    17

In [15]:
df_scans.to_csv(r'D:\vm_exchange\PhD_Data\GantnerValidation\scan_automatic_matching.csv', index=False, sep=';')